In [3]:
import torch
import random 
import numpy as np 
from tqdm import tqdm 
from scipy.spatial.distance import cdist, cosine
from scipy.optimize import linear_sum_assignment
from utils.corrupt_graph import remove_edge, remove_node, add_edge, add_node
from utils.query_machine import get_candidates
from python_emb import *
import torch.nn.functional as F
from scipy.spatial.distance import cosine
from scipy.stats import pearsonr, spearmanr
from collections import defaultdict

In [4]:
import time

In [15]:
def select_label_class(future):
    labels = list(future.keys())
    values = [max(len(future[i][0]), len(future[i][1])) for i in labels]
    selected_label = labels[np.argmin(np.array(values))]
    return selected_label, future[selected_label]

def select_vertex(setG, g):
    listG = list(setG)
    values = [g.degree(i) for i in setG]
    return listG[np.argmax(np.array(values))]

def mcs(g,h):
    stime = time.time()
    labelset_g = defaultdict(set)
    labelset_h = defaultdict(set)
    for node in g.nodes:
        labelset_g[g.nodes[node]['label']].add(node)
    for node in h.nodes:
        labelset_h[h.nodes[node]['label']].add(node)

    future = {}
    for label in set(labelset_g.keys()).intersection(labelset_h.keys()):
        future[str(label)+'-'] = [labelset_g[label], labelset_h[label]]
    M, incumbent = {}, {}

    def search(future):
        if time.time() - stime > 100:
            return False
        nonlocal incumbent, M, g,h
        if len(M) > len(incumbent):
            incumbent = M.copy()
        bound = len(M) + sum([ min(len(value[0]), len(value[1])) for value in future.values()])
        if bound <= len(incumbent):
            return 
        if len(future)==0:
            return
        selected_label, (setG, setH) = select_label_class(future)
        v = select_vertex(setG, g)
        for w in setH:
            future_ = {}
            for cur_label, (setG_, setH_) in future.items():
                setG__ = (setG_.intersection([node for node in g.neighbors(v)])).difference([v])
                setH__ = (setH_.intersection([node for node in h.neighbors(w)])).difference([w])
                if len(setG__) != 0 and len(setH__) != 0:
                    future_[cur_label+'1'] = [setG__, setH__]

                setG__ = (setG_.intersection([node for node in g.nodes if node not in g.neighbors(v)] )).difference([v])
                setH__ = (setH_.intersection([node for node in h.nodes if node not in h.neighbors(w)] )).difference([w])
                if len(setG__) != 0 and len(setH__) != 0:
                    future_[cur_label+'0'] = [setG__, setH__]
            M[v]=w
            search(future_)
            del M[v]
        setG_ = setG.difference([v])
        del future[selected_label]
        if len(setG_) > 0:
            future[selected_label] = [setG_, setH]
        search(future)
        return True
    
    if search(future):
        return incumbent
    return False

In [6]:
# for dataset in ['yeast', 'human', 'cora', 'citeseer', 'pubmed', 'wordnet']:
#     dim=4
#     ori_graph_data = load_data('./dataspace/graph/{}/{}'.format(dataset, dataset), supervised=False, max_degree=5, multiclass=False, use_random_walks=False)

#     ori_emb_model = SupervisedGraphSage(ori_graph_data.raw_feats.shape[1], dim, ori_graph_data.num_class)
#     ori_emb_model = ori_emb_model.cuda()
#     ori_emb_model.load_state_dict(torch.load('model_dim/{}_sup_20_dim{}.pt'.format(dataset, dim)))
#     ori_emb_model.set_params(ori_graph_data.full_adj, ori_graph_data.deg, ori_graph_data.feats)
#     ori_emb_model.eval()

#     ori_graph_emb = F.normalize(ori_emb_model.aggregator(list(range(ori_graph_data.raw_feats.shape[0]))), dim = 1)
#     ori_graph_emb = ori_graph_emb.detach().cpu().numpy()


#     for i, node in enumerate(ori_graph_data.G.nodes):
#         ori_graph_data.G.nodes[node]['label'] = ori_graph_data.multi2single_label[tuple(ori_graph_data.G.nodes[node]['label'])]

#     query_machine = GraphQuery(ori_emb_model, 
#             ori_graph_emb,
#             ori_graph_data.G,
#             ori_graph_data.id_map,
#             ori_graph_data.feats,
#             ori_graph_data.raw_feats,
#             ori_graph_data.full_adj,
#             ori_graph_data.deg)

#     def get_embedding_subgraph(subgraph):
#         sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(subgraph)
#         embedding_subgraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
#         embedding_subgraph = embedding_subgraph.detach().cpu().numpy()

#         return embedding_subgraph, sub_id_map


#     mcs2emb = defaultdict(list)
#     GRAPH_SIZE = 15
#     graphs = []
#     graph_embeddings = []
#     for i, core_node in tqdm(enumerate(query_machine.ori_graph.nodes)):
#         biggraph = query_machine.create_subgraph_from_core(core_node, GRAPH_SIZE)
#         if len(biggraph) != GRAPH_SIZE or not nx.is_connected(biggraph):
#             continue
#         sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(biggraph)

#         embedding_biggraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
#         embedding_biggraph = embedding_biggraph.detach().cpu().numpy().mean(0)
#         if not np.isnan(embedding_biggraph).any():
#             graphs.append(biggraph)
#             graph_embeddings.append(embedding_biggraph)
#     n=len(graphs)
#     edit_distances = []
#     emb_distances = []
#     edit_distances_tuple = []
#     for _ in tqdm(range(5000)):
#         i, j = random.randint(0, n-1),random.randint(0, n-1)
#         graph1 = graphs[i]
#         graph2 = graphs[j]
#         edit_distance = len(mcs(graph1, graph2))
#         edit_distances.append(edit_distance)
#         edit_distances_tuple.append( ( (i,j) , edit_distance) )
#         emb_distances.append( ((graph_embeddings[i] - graph_embeddings[j])**2).sum() )
#     print(Counter(edit_distances))
#     print(dataset, dim, spearmanr(edit_distances, emb_distances), pearsonr(edit_distances,emb_distances))
#     for dim in [8, 16, 32, 64, 128]:
#         ori_graph_data = load_data('./dataspace/graph/{}/{}'.format(dataset, dataset), supervised=False, max_degree=5, multiclass=False, use_random_walks=False)

#         ori_emb_model = SupervisedGraphSage(ori_graph_data.raw_feats.shape[1], dim, ori_graph_data.num_class)
#         ori_emb_model = ori_emb_model.cuda()
#         ori_emb_model.load_state_dict(torch.load('model_dim/{}_sup_20_dim{}.pt'.format(dataset, dim)))
#         ori_emb_model.set_params(ori_graph_data.full_adj, ori_graph_data.deg, ori_graph_data.feats)
#         ori_emb_model.eval()

#         ori_graph_emb = F.normalize(ori_emb_model.aggregator(list(range(ori_graph_data.raw_feats.shape[0]))), dim = 1)
#         ori_graph_emb = ori_graph_emb.detach().cpu().numpy()


#         for i, node in enumerate(ori_graph_data.G.nodes):
#             ori_graph_data.G.nodes[node]['label'] = ori_graph_data.multi2single_label[tuple(ori_graph_data.G.nodes[node]['label'])]

#         query_machine = GraphQuery(ori_emb_model, 
#                 ori_graph_emb,
#                 ori_graph_data.G,
#                 ori_graph_data.id_map,
#                 ori_graph_data.feats,
#                 ori_graph_data.raw_feats,
#                 ori_graph_data.full_adj,
#                 ori_graph_data.deg)

#         def get_embedding_subgraph(subgraph):
#             sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(subgraph)
#             embedding_subgraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
#             embedding_subgraph = embedding_subgraph.detach().cpu().numpy()

#             return embedding_subgraph, sub_id_map

#         graph_embeddings = []
#         for biggraph in tqdm(graphs):
#             sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(biggraph)

#             embedding_biggraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
#             embedding_biggraph = embedding_biggraph.detach().cpu().numpy().mean(0)
#             graph_embeddings.append(embedding_biggraph)
                
#         n=len(graphs)
#         edit_distances = []
#         emb_distances = []
#         for (i, j), edit_distance in tqdm(edit_distances_tuple):
#             edit_distances.append(edit_distance)
#             emb_distances.append( ((graph_embeddings[i] - graph_embeddings[j])**2).sum() )
#         print(dataset, dim, spearmanr(edit_distances, emb_distances), pearsonr(edit_distances,emb_distances))

Set max degree to 5
-----------------------------------------------
Loading data:
Loading graph data from ./dataspace/graph/yeast/yeast-G.json
Removed 0 nodes that lacked proper annotations due to networkx versioning issues
File loaded successfully
Loading feature from ./dataspace/graph/yeast/yeast-G.json
File loaded successfully
Loading classmap data from ./dataspace/graph/yeast/yeast-class_map.json
File loaded successfully
Loaded data.. now preprocessing..
Use original edges
Generate train edges
Number of training edges: 12519
Preprocessing finished, graph info:
Name: yeast
Type: Graph
Number of nodes: 3101
Number of edges: 12519
Average degree:   8.0742


2250it [00:06, 342.00it/s]


KeyboardInterrupt: 

In [11]:
for dataset in ['wordnet']:
    dim=4
    ori_graph_data = load_data('./dataspace/graph/{}/{}'.format(dataset, dataset), supervised=False, max_degree=5, multiclass=False, use_random_walks=False)

    ori_emb_model = SupervisedGraphSage(ori_graph_data.raw_feats.shape[1], dim, ori_graph_data.num_class)
    ori_emb_model = ori_emb_model.cuda()
    ori_emb_model.load_state_dict(torch.load('model_dim/{}_sup_20_dim{}.pt'.format(dataset, dim)))
    ori_emb_model.set_params(ori_graph_data.full_adj, ori_graph_data.deg, ori_graph_data.feats)
    ori_emb_model.eval()

    ori_graph_emb = F.normalize(ori_emb_model.aggregator(list(range(ori_graph_data.raw_feats.shape[0]))), dim = 1)
    ori_graph_emb = ori_graph_emb.detach().cpu().numpy()


    for i, node in enumerate(ori_graph_data.G.nodes):
        ori_graph_data.G.nodes[node]['label'] = ori_graph_data.multi2single_label[tuple(ori_graph_data.G.nodes[node]['label'])]

    query_machine = GraphQuery(ori_emb_model, 
            ori_graph_emb,
            ori_graph_data.G,
            ori_graph_data.id_map,
            ori_graph_data.feats,
            ori_graph_data.raw_feats,
            ori_graph_data.full_adj,
            ori_graph_data.deg)

    def get_embedding_subgraph(subgraph):
        sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(subgraph)
        embedding_subgraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
        embedding_subgraph = embedding_subgraph.detach().cpu().numpy()

        return embedding_subgraph, sub_id_map


    mcs2emb = defaultdict(list)
    GRAPH_SIZE = 15
    graphs = []
    graph_embeddings = []
    for i, core_node in tqdm(enumerate(query_machine.ori_graph.nodes)):
        biggraph = query_machine.create_subgraph_from_core(core_node, GRAPH_SIZE)
        if len(biggraph) != GRAPH_SIZE or not nx.is_connected(biggraph):
            continue
        sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(biggraph)

        embedding_biggraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
        embedding_biggraph = embedding_biggraph.detach().cpu().numpy().mean(0)
        if not np.isnan(embedding_biggraph).any():
            graphs.append(biggraph)
            graph_embeddings.append(embedding_biggraph)
    n=len(graphs)
    edit_distances = []
    emb_distances = []
    edit_distances_tuple = []
    while len(emb_distances)<5000:
        i, j = random.randint(0, n-1),random.randint(0, n-1)
        graph1 = graphs[i]
        graph2 = graphs[j]
        mcs_graph = mcs(graph1, graph2)
        if mcs_graph:
            edit_distance = len(mcs_graph)
            edit_distances.append(edit_distance)
            edit_distances_tuple.append( ( (i,j) , edit_distance) )
            emb_distances.append( ((graph_embeddings[i] - graph_embeddings[j])**2).sum() )
            print(len(emb_distances),end='\r')
    print(Counter(edit_distances))
    print(dataset, dim, spearmanr(edit_distances, emb_distances), pearsonr(edit_distances,emb_distances))
    for dim in [8, 16, 32, 64, 128]:
        ori_graph_data = load_data('./dataspace/graph/{}/{}'.format(dataset, dataset), supervised=False, max_degree=5, multiclass=False, use_random_walks=False)

        ori_emb_model = SupervisedGraphSage(ori_graph_data.raw_feats.shape[1], dim, ori_graph_data.num_class)
        ori_emb_model = ori_emb_model.cuda()
        ori_emb_model.load_state_dict(torch.load('model_dim/{}_sup_20_dim{}.pt'.format(dataset, dim)))
        ori_emb_model.set_params(ori_graph_data.full_adj, ori_graph_data.deg, ori_graph_data.feats)
        ori_emb_model.eval()

        ori_graph_emb = F.normalize(ori_emb_model.aggregator(list(range(ori_graph_data.raw_feats.shape[0]))), dim = 1)
        ori_graph_emb = ori_graph_emb.detach().cpu().numpy()


        for i, node in enumerate(ori_graph_data.G.nodes):
            ori_graph_data.G.nodes[node]['label'] = ori_graph_data.multi2single_label[tuple(ori_graph_data.G.nodes[node]['label'])]

        query_machine = GraphQuery(ori_emb_model, 
                ori_graph_emb,
                ori_graph_data.G,
                ori_graph_data.id_map,
                ori_graph_data.feats,
                ori_graph_data.raw_feats,
                ori_graph_data.full_adj,
                ori_graph_data.deg)

        def get_embedding_subgraph(subgraph):
            sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(subgraph)
            embedding_subgraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
            embedding_subgraph = embedding_subgraph.detach().cpu().numpy()

            return embedding_subgraph, sub_id_map

        graph_embeddings = []
        for biggraph in tqdm(graphs):
            sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(biggraph)

            embedding_biggraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
            embedding_biggraph = embedding_biggraph.detach().cpu().numpy().mean(0)
            graph_embeddings.append(embedding_biggraph)
                
        n=len(graphs)
        edit_distances = []
        emb_distances = []
        for (i, j), edit_distance in tqdm(edit_distances_tuple):
            edit_distances.append(edit_distance)
            emb_distances.append( ((graph_embeddings[i] - graph_embeddings[j])**2).sum() )
        print(dataset, dim, spearmanr(edit_distances, emb_distances), pearsonr(edit_distances,emb_distances))

Set max degree to 5
-----------------------------------------------
Loading data:
Loading graph data from ./dataspace/graph/wordnet/wordnet-G.json
Removed 0 nodes that lacked proper annotations due to networkx versioning issues
File loaded successfully
Loading feature from ./dataspace/graph/wordnet/wordnet-G.json
File loaded successfully
Loading classmap data from ./dataspace/graph/wordnet/wordnet-class_map.json
File loaded successfully
Loaded data.. now preprocessing..
Use original edges
Generate train edges
Number of training edges: 127124
Preprocessing finished, graph info:
Name: wordnet
Type: Graph
Number of nodes: 82670
Number of edges: 127124
Average degree:   3.0755


82670it [03:08, 438.58it/s] 


KeyboardInterrupt: 

In [16]:
n=len(graphs)
edit_distances = []
emb_distances = []
edit_distances_tuple = []
while len(emb_distances)<5000:
    i, j = random.randint(0, n-1),random.randint(0, n-1)
    graph1 = graphs[i]
    graph2 = graphs[j]
    mcs_graph = mcs(graph1, graph2)
    if mcs_graph:
        edit_distance = len(mcs_graph)
        edit_distances.append(edit_distance)
        edit_distances_tuple.append( ( (i,j) , edit_distance) )
        emb_distances.append( ((graph_embeddings[i] - graph_embeddings[j])**2).sum() )
        print(len(emb_distances),end='\r')
print(Counter(edit_distances))
print(dataset, dim, spearmanr(edit_distances, emb_distances), pearsonr(edit_distances,emb_distances))
for dim in [8, 16, 32, 64, 128]:
    ori_graph_data = load_data('./dataspace/graph/{}/{}'.format(dataset, dataset), supervised=False, max_degree=5, multiclass=False, use_random_walks=False)

    ori_emb_model = SupervisedGraphSage(ori_graph_data.raw_feats.shape[1], dim, ori_graph_data.num_class)
    ori_emb_model = ori_emb_model.cuda()
    ori_emb_model.load_state_dict(torch.load('model_dim/{}_sup_20_dim{}.pt'.format(dataset, dim)))
    ori_emb_model.set_params(ori_graph_data.full_adj, ori_graph_data.deg, ori_graph_data.feats)
    ori_emb_model.eval()

    ori_graph_emb = F.normalize(ori_emb_model.aggregator(list(range(ori_graph_data.raw_feats.shape[0]))), dim = 1)
    ori_graph_emb = ori_graph_emb.detach().cpu().numpy()


    for i, node in enumerate(ori_graph_data.G.nodes):
        ori_graph_data.G.nodes[node]['label'] = ori_graph_data.multi2single_label[tuple(ori_graph_data.G.nodes[node]['label'])]

    query_machine = GraphQuery(ori_emb_model, 
            ori_graph_emb,
            ori_graph_data.G,
            ori_graph_data.id_map,
            ori_graph_data.feats,
            ori_graph_data.raw_feats,
            ori_graph_data.full_adj,
            ori_graph_data.deg)

    def get_embedding_subgraph(subgraph):
        sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(subgraph)
        embedding_subgraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
        embedding_subgraph = embedding_subgraph.detach().cpu().numpy()

        return embedding_subgraph, sub_id_map

    graph_embeddings = []
    for biggraph in tqdm(graphs):
        sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(biggraph)

        embedding_biggraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
        embedding_biggraph = embedding_biggraph.detach().cpu().numpy().mean(0)
        graph_embeddings.append(embedding_biggraph)

    n=len(graphs)
    edit_distances = []
    emb_distances = []
    for (i, j), edit_distance in tqdm(edit_distances_tuple):
        edit_distances.append(edit_distance)
        emb_distances.append( ((graph_embeddings[i] - graph_embeddings[j])**2).sum() )
    print(dataset, dim, spearmanr(edit_distances, emb_distances), pearsonr(edit_distances,emb_distances))

Counter({12: 1547, 11: 1108, 10: 504, 9: 383, 13: 320, 8: 227, 7: 177, 6: 160, 5: 136, 4: 133, 3: 108, 2: 99, 1: 88, 14: 10})
wordnet 4 SpearmanrResult(correlation=-0.8612662991422443, pvalue=0.0) (-0.8996118552146768, 0.0)
Set max degree to 5
-----------------------------------------------
Loading data:
Loading graph data from ./dataspace/graph/wordnet/wordnet-G.json
Removed 0 nodes that lacked proper annotations due to networkx versioning issues
File loaded successfully
Loading feature from ./dataspace/graph/wordnet/wordnet-G.json
File loaded successfully
Loading classmap data from ./dataspace/graph/wordnet/wordnet-class_map.json
File loaded successfully
Loaded data.. now preprocessing..
Use original edges
Generate train edges
Number of training edges: 127124
Preprocessing finished, graph info:
Name: wordnet
Type: Graph
Number of nodes: 82670
Number of edges: 127124
Average degree:   3.0755


100%|██████████| 5000/5000 [00:00<00:00, 227459.30it/s]


wordnet 8 SpearmanrResult(correlation=-0.8630103241999257, pvalue=0.0) (-0.9457853482254593, 0.0)
Set max degree to 5
-----------------------------------------------
Loading data:
Loading graph data from ./dataspace/graph/wordnet/wordnet-G.json
Removed 0 nodes that lacked proper annotations due to networkx versioning issues
File loaded successfully
Loading feature from ./dataspace/graph/wordnet/wordnet-G.json
File loaded successfully
Loading classmap data from ./dataspace/graph/wordnet/wordnet-class_map.json
File loaded successfully
Loaded data.. now preprocessing..
Use original edges
Generate train edges
Number of training edges: 127124
Preprocessing finished, graph info:
Name: wordnet
Type: Graph
Number of nodes: 82670
Number of edges: 127124
Average degree:   3.0755


100%|██████████| 5000/5000 [00:00<00:00, 227326.16it/s]


wordnet 16 SpearmanrResult(correlation=-0.8634712339192067, pvalue=0.0) (-0.9453260580121946, 0.0)
Set max degree to 5
-----------------------------------------------
Loading data:
Loading graph data from ./dataspace/graph/wordnet/wordnet-G.json
Removed 0 nodes that lacked proper annotations due to networkx versioning issues
File loaded successfully
Loading feature from ./dataspace/graph/wordnet/wordnet-G.json
File loaded successfully
Loading classmap data from ./dataspace/graph/wordnet/wordnet-class_map.json
File loaded successfully
Loaded data.. now preprocessing..
Use original edges
Generate train edges
Number of training edges: 127124
Preprocessing finished, graph info:
Name: wordnet
Type: Graph
Number of nodes: 82670
Number of edges: 127124
Average degree:   3.0755


...

wordnet 32 SpearmanrResult(correlation=-0.8622446078404036, pvalue=0.0) (-0.9412014976257482, 0.0)
Set max degree to 5
-----------------------------------------------
Loading data:
Loading graph data from ./dataspace/graph/wordnet/wordnet-G.json
Removed 0 nodes that lacked proper annotations due to networkx versioning issues
File loaded successfully
Loading feature from ./dataspace/graph/wordnet/wordnet-G.json
File loaded successfully
Loading classmap data from ./dataspace/graph/wordnet/wordnet-class_map.json
File loaded successfully
Loaded data.. now preprocessing..
Use original edges
Generate train edges
Number of training edges: 127124
Preprocessing finished, graph info:
Name: wordnet
Type: Graph
Number of nodes: 82670
Number of edges: 127124
Average degree:   3.0755


100%|██████████| 5000/5000 [00:00<00:00, 210272.42it/s]


wordnet 64 SpearmanrResult(correlation=-0.8615557754842785, pvalue=0.0) (-0.9417961211792564, 0.0)
Set max degree to 5
-----------------------------------------------
Loading data:
Loading graph data from ./dataspace/graph/wordnet/wordnet-G.json
Removed 0 nodes that lacked proper annotations due to networkx versioning issues
File loaded successfully
Loading feature from ./dataspace/graph/wordnet/wordnet-G.json
File loaded successfully
Loading classmap data from ./dataspace/graph/wordnet/wordnet-class_map.json
File loaded successfully
Loaded data.. now preprocessing..
Use original edges
Generate train edges
Number of training edges: 127124
Preprocessing finished, graph info:
Name: wordnet
Type: Graph
Number of nodes: 82670
Number of edges: 127124
Average degree:   3.0755


100%|██████████| 5000/5000 [00:00<00:00, 205116.49it/s]

wordnet 128 SpearmanrResult(correlation=-0.8619479520066587, pvalue=0.0) (-0.9385003850698971, 0.0)
